### Setting paths and imports

In [2]:
import sys
import os
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
module_path

'/NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL'

In [3]:
os.chdir(module_path)

In [4]:
# from src.evaluation.evaluator_manager import EvaluatorManager
# from src.evaluation.future.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.utils.context import Context
from src.data_analysis.future.data_analyzer import DataAnalyzer as data_analyzer

from src.evaluation.future.evaluator_manager_triplets import EvaluatorManager

In [5]:
# config_f_name = 'ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_Ensemble-Bidirectional.jsonc'
# config_f_name = 'ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_dl-explainer-selector.jsonc'
config_f_name = 'ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_RSGG.jsonc'
config_f_name = 'ensembles/triplets/TCR-TCO/il-explainer-selector.jsonc'

### Evaluating the explainer

In [5]:
config_path = os.path.join(module_path, 'lab', 'config', config_f_name)
runno = 1
    
print(f"Generating context for: {config_path}")
context = Context.get_context(config_path)
context.run_number = runno

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info("Creating the evaluation manager....................................")

context.logger.info("Creating the evaluators......................................................")
eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)
eval_manager.evaluate()

Generating context for: /NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/lab/config/ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_RSGG.jsonc
2024-08-05 10:40:59,565374076 | INFO | 1060102 - Executing: /NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/lab/config/ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_RSGG.jsonc Run: 1
2024-08-05 10:40:59,565374092 | INFO | 1060102 - Creating the evaluation manager....................................
2024-08-05 10:40:59,565374106 | INFO | 1060102 - Creating the evaluators......................................................
2024-08-05 10:40:59,565374240 | INFO | 1060102 - Loading: TreeCyclesRand-59b4a4ad5355de4e6189d393902be53c
2024-08-05 10:41:00,565374382 | INFO | 1060102 - Created: TreeCyclesRand-59b4a4ad5355de4e6189d393902be53c
2024-08-05 10:41:00,565374482 | INFO | 1060102 - Creating: TreeCyclesOracle-dad4bc4709835728d485620b5506eeaf
2024-08-05 10:41:00,565374505 | INFO | 1060102 - TreeCyclesOracle trained in: 5.0067901611328125e-06 secs
2024-08-05

/NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/src/legacy/explainer/generative/gans/graph/res_gen.py:64: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  edge_weights = torch.add(edge_weights, rebuild_adj_matrix(len(node_features), edge_list, edge_attr.T, self.device))
/NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/src/legacy/explainer/generative/gans/graph/model.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = torch.mean(self.loss_fn(y_pred.squeeze().double(), y_batch.double()) * torch.tensor(oracle_scores, dtype=torch.float))


2024-08-05 10:41:01,565375585 | INFO | 1060102 - Epoch 0	 Loss_D =  0.6316	 Loss_G =  3.3251
2024-08-05 10:41:01,565375638 | INFO | 1060102 - Epoch 1	 Loss_D =  0.8031	 Loss_G =  2.7556
2024-08-05 10:41:01,565375700 | INFO | 1060102 - Epoch 2	 Loss_D =  0.7165	 Loss_G =  2.3259
2024-08-05 10:41:01,565375772 | INFO | 1060102 - Epoch 3	 Loss_D =  0.7115	 Loss_G =  2.3962
2024-08-05 10:41:01,565375863 | INFO | 1060102 - Epoch 4	 Loss_D =  0.7200	 Loss_G =  2.6502
2024-08-05 10:41:01,565375925 | INFO | 1060102 - Epoch 5	 Loss_D =  0.6880	 Loss_G =  2.7717
2024-08-05 10:41:01,565375978 | INFO | 1060102 - Epoch 6	 Loss_D =  0.5988	 Loss_G =  2.3553
2024-08-05 10:41:01,565376019 | INFO | 1060102 - Epoch 7	 Loss_D =  0.7099	 Loss_G =  1.9734
2024-08-05 10:41:01,565376063 | INFO | 1060102 - Epoch 8	 Loss_D =  0.7350	 Loss_G =  2.1104
2024-08-05 10:41:01,565376107 | INFO | 1060102 - Epoch 9	 Loss_D =  0.5318	 Loss_G =  1.9725
2024-08-05 10:41:01,565376177 | INFO | 1060102 - Epoch 10	 Loss_D =  0

### Aggregating the stats

In [6]:
results_path = os.path.join(module_path, 'lab', 'output', 'results')
stats_file_path = os.path.join(module_path, 'lab', 'stats', 'results.csv')
res = data_analyzer.create_aggregated_dataframe(results_path)
res.to_csv(stats_file_path)
res

,scope,dataset,oracle,explainer,Runtime,Runtime-std,GraphEditDistance,GraphEditDistance-std,Correctness,Correctness-std,OracleCalls,OracleCalls-std,OracleAccuracy,OracleAccuracy-std,Sparsity,Sparsity-std,Fidelity,Fidelity-std,Instability,Instability-std
0,xai-ens,TreeCyclesRand-59b4a4ad5355de4e6189d393902be53c,TreeCyclesOracle-dad4bc4709835728d485620b5506eeaf,ObliviousBidirectionalSearchExplainer,0.238584,0.202896,1.300233,0.128174,0.976282,0.036268,353.110256,124.308445,1.0,0.0,0.030952,0.002986,0.976282,0.036268,1.013930,0.006781
1,xai-ens,TreeCyclesRand-59b4a4ad5355de4e6189d393902be53c,TreeCyclesOracle-dad4bc4709835728d485620b5506eeaf,RSGG,3.218562,1.027877,14.472917,0.192149,0.775641,0.091915,199.456410,79.269073,1.0,0.0,0.346698,0.004456,0.775641,0.091915,0.710149,0.034786
2,xai-ens,TreeCyclesRand-59b4a4ad5355de4e6189d393902be53c,TreeCyclesOracle-dad4bc4709835728d485620b5506eeaf,IRandExplainer,0.070726,0.019783,1.000000,0.000000,0.453205,0.029522,67.378205,3.771363,1.0,0.0,0.024092,0.000014,0.453205,0.029522,0.993434,0.002098
3,xai-ens,TreeCyclesRand-266bb3721206805597afc8c4184fe0ab,TreeCyclesOracle-8f8e216deff127e0218ff13e1490c1a2,ExplanationTopSelect,7.390612,2.761418,1.212821,0.109756,1.000000,0.000000,1395.514744,158.585613,1.0,0.0,0.028903,0.002551,1.000000,0.000000,1.018937,0.008537
4,xai-ens,TreeCyclesRand-266bb3721206805597afc8c4184fe0ab,TreeCyclesOracle-8f8e216deff127e0218ff13e1490c1a2,ExplanationRandom,8.085943,3.334642,0.500000,0.000000,0.484615,0.030769,1532.914744,392.759206,1.0,0.0,0.012048,0.000000,0.484615,0.030769,0.984513,0.001554
5,xai-ens,TreeCyclesRand-266bb3721206805597afc8c4184fe0ab,TreeCyclesOracle-8f8e216deff127e0218ff13e1490c1a2,ExplanationStochasticAggregator,6.649459,3.657311,6.554808,0.412271,1.000000,0.000000,1457.980128,270.717506,1.0,0.0,0.156607,0.009845,1.000000,0.000000,0.904825,0.017122
6,xai-ens,TreeCyclesRand-266bb3721206805597afc8c4184fe0ab,TreeCyclesOracle-8f8e216deff127e0218ff13e1490c1a2,ExplanationFrequency,6.346897,1.898049,0.000000,0.000000,0.000000,0.000000,1494.597436,443.060097,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.953253,0.013228
7,xai-ens,TreeCyclesRand-266bb3721206805597afc8c4184fe0ab,TreeCyclesOracle-8f8e216deff127e0218ff13e1490c1a2,ExplanationMultiCriteriaAggregator,7.131712,2.029982,1.212821,0.109756,1.000000,0.000000,1366.149359,184.527446,1.0,0.0,0.028903,0.002551,1.000000,0.000000,1.020017,0.007832
8,xai-ens,TreeCyclesRand-266bb3721206805597afc8c4184fe0ab,TreeCyclesOracle-8f8e216deff127e0218ff13e1490c1a2,ExplanationIntersection,6.703976,3.227985,0.000000,0.000000,0.000000,0.000000,1429.841026,459.369443,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.975632,0.000897
9,xai-ens,TreeCyclesRand-266bb3721206805597afc8c4184fe0ab,TreeCyclesOracle-8f8e216deff127e0218ff13e1490c1a2,ExplanationBidirectionalSearch,7.458365,2.390505,1.192949,0.103698,1.000000,0.000000,1381.345513,282.555726,1.0,0.0,0.028424,0.002409,1.000000,0.000000,1.019050,0.006820


### Visualizing individual instances

In [ ]:
evaluator = eval_manager.evaluators[0]
evaluator

In [ ]:
exp = evaluator.explanations[2]
og_inst = exp.input_instance
cf_inst = exp.counterfactual_instances[0]

In [ ]:

changes = data_analyzer.get_cf_changes(og_inst, cf_inst, False)

In [ ]:
common_edges = changes['common edges']
added_edges = changes['added edges']
removed_edges = changes['removed edges']
common_nodes = changes['common nodes']
added_nodes = changes['added nodes']
removed_nodes = changes['removed nodes']

print(f'added edges: {added_edges}')
print(f'removed_edges: {removed_edges}')
print(f'added nodes: {added_nodes}')
print(f'removed nodes: {removed_nodes}')

In [ ]:
pos = data_analyzer.draw_graph(og_inst)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, cf_inst, position=pos)